In [3]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package list

In [5]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-16 05:03:19--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-03-16 05:03:20 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
pet_products_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="mm/dd/yyyy")

# Show DataFrame
pet_products_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [8]:
# Get the number of rows in the DataFrame.
pet_products_df.count()

2643619

In [9]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = pet_products_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789| 2015-08-31|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483| 2015-08-31|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658| 2015-08-31|
|R33GITXNUF1AD4|   33930128|B00BOEXWFG|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|   43534290|B001HBBQKY|     420905252| 2015-08

In [10]:
# Get the number of rows in the DataFrame
review_id_df.count()

2643619

In [11]:
# Describe the datatypes in the DataFrame
review_id_df.describe()

DataFrame[summary: string, review_id: string, customer_id: string, product_id: string, product_parent: string, review_date: string]

In [12]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = pet_products_df.select(["product_id", "product_title"])
products_df.show(20)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Q0K9604|(8-Pack) EZwhelp ...|
|B00MBW5O9W|Warren Eckstein's...|
|B0084OHUIO|Tyson's True Chew...|
|B001GS71K2|Soft Side Pet Cra...|
|B004ABH1LG|EliteField 3-Door...|
|B00AX0LFM4|Carlson 68-Inch W...|
|B00DQFZGZ0|Dog Seat Cover Wi...|
|B00DIRF9US|The Bird Catcher ...|
|B00JRCBFUG|Cat Bed - Purrfec...|
|B000L3XYZ4|PetSafe Drinkwell...|
|B00BOEXWFG|Contech ZenDog Ca...|
|B001HBBQKY|Wellness Crunchy ...|
|B007O1FHB0|Rx Vitamins Essen...|
|B001P3NU30|Virbac C.E.T. Enz...|
|B00ZP6HS6S|Kitty Shack - 2 i...|
|B00IP05CUA|Wellness Kittles ...|
|B001U8Y598|OmniPet Anti-Ant ...|
|B011AY4JWO|K9KONNECTION [New...|
|B00DDSHE5A|SUNSEED COMPANY 3...|
|B00PJW5OR8|CXB1983(TM)Cute P...|
+----------+--------------------+
only showing top 20 rows



In [13]:
print(products_df.count())

2643619


In [14]:
products_df = products_df.dropDuplicates(["product_id", "product_title"])

In [15]:
# Verify rows dropped and get the new number of rows in the DataFrame
products_df.count()


239343

In [16]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

customers_df = pet_products_df.groupby("customer_id").agg({"customer_id": "count"})
customers_df.show(20)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   10270641|                 1|
|   18365872|                 1|
|   16711087|                 1|
|   10742726|                 2|
|   41169638|                 1|
|   43622307|                 1|
|   24540309|                 2|
|   28258386|                 1|
|   35329257|                 2|
|   14552054|                 1|
|   14529507|                 5|
|   45392827|                 5|
|   47282953|                 1|
|    8201930|                 1|
|   20109760|                 2|
|   16405801|                 4|
|   15056685|                21|
|   20840575|                 2|
|   39048303|                 1|
|    5596610|                 1|
+-----------+------------------+
only showing top 20 rows



In [17]:
# Get total count of reviews per customer_id
from pyspark.sql.functions import desc
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.orderBy(desc("customer_count")).show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   22500722|           258|
|   31511151|           236|
|   16053845|           228|
|   25659082|           210|
|   23375624|           200|
|   43856165|           198|
|   51175061|           163|
|   37446839|           160|
|   16540594|           150|
|   52263513|           134|
|   16916667|           134|
|   33315159|           134|
|   18164293|           132|
|   18539854|           132|
|   29676361|           130|
|   39154578|           121|
|   16052325|           120|
|   38352624|           119|
|   50991253|           115|
|   42329785|           115|
+-----------+--------------+
only showing top 20 rows



In [18]:
# Get the number of rows in the DataFrame
customers_df.count()

1415190

In [19]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = pet_products_df.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
|R33GITXNUF1AD4|          2|            0|          0|   N|
|R1H7AVM81TAYRV|          1|            2|          2|   N|
|R2ZOYAQZNNZZWV|          5|            0|          0|   N|
|R2FN1H3CGW6J8H|          1|            

In [20]:
# Get the number of rows in the DataFrame
vine_df.count()

2643619

In [23]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cyjjgths7h9i.us-west-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [27]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_df", mode=mode, properties=config)

In [27]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="products_df", mode=mode, properties=config)

In [27]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="vine_df", mode=mode, properties=config)

In [27]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table="vine_df", mode=mode, properties=config)